# Load Packages

In [1]:
%load_ext autoreload
%autoreload 2

from RT_arduino import Arduino

# Initialize connection with Arduino

Note that `keys_dout` can be removed from below if not using digital outputs.  Likewise, `keys_din` can be removed from below if not using digital inputs.

**If utilizing digital inputs, ports must also be set in line 7 of `ArduinoSketch.ino`**

In [ ]:
ard = Arduino(
    port = '/dev/cu.usbmodem1101',
    keys_dout = {
        'light' : 13
     },
    keys_din = {
        'beam1' : 4,
        'beam2' : 5,
    }
)
ard.initialize()

# Deliver signal to turn on light on Arduino board

#### Turn on light

In [ ]:
ard.digitalHigh('light')

#### Turn off light

In [ ]:
ard.digitalLow('light')

#### Turn on light for 5 seconds

In [ ]:
ard.digitalHigh('light', hold=5)

# Create simple task where beam break on active port triggers arduino led

The following assumes digital input named beam1 is connected to Arduino.  
Whenever beam is broken led is lit up.  
Additionally, inputs from both ports are recorded in dataframe.

In [ ]:
import pandas as pd
import time
from threading import Thread

def task(duration=3, active='beam1'):
    
    Tstart = time.time() #start time
    ard.flushInput() #flush any existing input data before starting
    ##global df
    df = pd.DataFrame(columns = ['din','state','time']) #initialize data frame

    while time.time() - Tstart < duration:
        if not ard.q_inputs.empty():
            i = ard.q_inputs.get()
            if i['din'] == ard.keys_din[active]:
                if i['state'] == 0: #beam broken
                    ard.digitalHigh('light')
                else: #beam unbroken
                    ard.digitalLow('light')
            df = pd.concat([df, pd.DataFrame(i, index=[0])])
    print(df)

Thread(target=task, kwargs=({'duration':10, 'active':'beam1'})).start()

# Close connection with Arduino

In [ ]:
ard.stop()